# Meta ML System Design: Overview & Mental Models

**Target Level:** Meta E5 → E6  
**Prerequisite:** Strong systems background, ML beginner  
**Interview Focus:** Ranking systems, Feed/Ads/Reels/Notifications

---

## What This Notebook Covers

This is your mental model for how Meta thinks about ML systems. Every system at Meta—Feed ranking, Ads, Reels recommendations, Notifications—follows the same fundamental architecture pattern. Master this pattern, and you can design any Meta ML system.

---

## 1. The Meta ML System Mental Model

### Everything is Ranking

At Meta scale, **every ML problem is fundamentally a ranking problem**:

| Product | What Gets Ranked | Objective |
|---------|------------------|----------|
| News Feed | Posts from friends, groups, pages | Maximize meaningful engagement |
| Ads | Advertiser creatives | Maximize revenue × relevance |
| Reels | Short videos | Maximize watch time + shares |
| Notifications | Push notifications | Maximize opens without annoyance |
| People You May Know | Friend suggestions | Maximize accepted connections |

Even "classification" problems (spam detection, hate speech) feed into ranking: a spam score becomes a penalty in the ranking formula.

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                     META ML SYSTEM CANONICAL ARCHITECTURE                     │
└─────────────────────────────────────────────────────────────────────────────┘

                              USER REQUEST
                                   │
                                   ▼
┌─────────────────────────────────────────────────────────────────────────────┐
│                         CANDIDATE GENERATION                                 │
│  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐        │
│  │  Friends'   │  │   Group     │  │   Page      │  │    Ads      │        │
│  │   Posts     │  │   Posts     │  │   Posts     │  │  Inventory  │        │
│  └─────────────┘  └─────────────┘  └─────────────┘  └─────────────┘        │
│                                                                             │
│  Goal: Reduce billions → thousands (cheap retrieval, ~10ms)                 │
│  Methods: Inverted index, embedding similarity, rule-based filters          │
└─────────────────────────────────────────────────────────────────────────────┘
                                   │
                                   ▼ ~1000-10000 candidates
┌─────────────────────────────────────────────────────────────────────────────┐
│                              FIRST-PASS RANKING                              │
│                                                                             │
│  Lightweight model (logistic regression, small neural net)                  │
│  Goal: Reduce thousands → hundreds (~50ms)                                  │
│  Features: User features + item features (precomputed)                      │
└─────────────────────────────────────────────────────────────────────────────┘
                                   │
                                   ▼ ~100-500 candidates
┌─────────────────────────────────────────────────────────────────────────────┐
│                              MAIN RANKING                                    │
│                                                                             │
│  Heavy model (deep neural network, multi-task learning)                     │
│  Goal: Precise ordering of top candidates (~100ms)                          │
│  Features: Dense features, cross features, context                          │
│                                                                             │
│  Predicts: P(click), P(like), P(comment), P(share), P(hide), P(report)     │
└─────────────────────────────────────────────────────────────────────────────┘
                                   │
                                   ▼ scored candidates
┌─────────────────────────────────────────────────────────────────────────────┐
│                              RE-RANKING / BLENDING                           │
│                                                                             │
│  Business rules, diversity injection, freshness boost                       │
│  Goal: Final slate optimization (~20ms)                                     │
│  Examples: No duplicate authors, mix content types, boost breaking news     │
└─────────────────────────────────────────────────────────────────────────────┘
                                   │
                                   ▼
                           FINAL RANKED LIST
                          (shown to user)
```

> **🧠 Meta Interview Signal**
>
> **Strong E5 Answer:** Explains the funnel clearly—candidate generation for recall, ranking for precision. Mentions latency budgets at each stage.
>
> **E6 Answer Adds:** Discusses why stages exist (cost/latency tradeoff), how models at each stage are trained differently, and what happens when one stage fails.
>
> **Common Pitfall:** Jumping straight to "use a transformer" without discussing the retrieval funnel. At Meta scale, you can't score billions of items with a heavy model.

## 2. The Ranking Formula: Combining Multiple Objectives

Meta's ranking systems don't optimize a single metric. They combine multiple predictions into a **value function**:

```
Score = w₁ × P(click) 
      + w₂ × P(like) 
      + w₃ × P(comment) 
      + w₄ × P(share) 
      + w₅ × P(long_watch) 
      - w₆ × P(hide) 
      - w₇ × P(report)
```

**Key Insight:** The weights (w₁, w₂, ...) are **policy decisions**, not learned. Product teams set these based on what engagement they want to encourage.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Simulate model predictions for 10 posts
np.random.seed(42)
n_posts = 10

posts = pd.DataFrame({
    'post_id': range(n_posts),
    'p_click': np.random.beta(2, 5, n_posts),      # Most posts: low-medium click prob
    'p_like': np.random.beta(1, 8, n_posts),       # Likes are rarer
    'p_comment': np.random.beta(1, 15, n_posts),   # Comments are rare
    'p_share': np.random.beta(1, 20, n_posts),     # Shares are very rare
    'p_hide': np.random.beta(1, 30, n_posts),      # Hides are rare (negative signal)
})

print("Model predictions for each post:")
print(posts.round(3))

In [ ]:
# Different weight configurations represent different product priorities

def compute_score(df, weights):
    """Compute ranking score given model predictions and weights."""
    return (
        weights['click'] * df['p_click'] +
        weights['like'] * df['p_like'] +
        weights['comment'] * df['p_comment'] +
        weights['share'] * df['p_share'] -
        weights['hide'] * df['p_hide']
    )

# Scenario 1: Engagement-focused (optimize clicks)
engagement_weights = {'click': 1.0, 'like': 0.5, 'comment': 0.3, 'share': 0.3, 'hide': 2.0}

# Scenario 2: Quality-focused (prioritize meaningful interactions)
quality_weights = {'click': 0.1, 'like': 0.5, 'comment': 2.0, 'share': 3.0, 'hide': 5.0}

# Scenario 3: Viral-focused (maximize shares)
viral_weights = {'click': 0.2, 'like': 0.2, 'comment': 0.5, 'share': 5.0, 'hide': 1.0}

posts['score_engagement'] = compute_score(posts, engagement_weights)
posts['score_quality'] = compute_score(posts, quality_weights)
posts['score_viral'] = compute_score(posts, viral_weights)

# Show how the same predictions lead to different rankings
print("Rankings under different objective weights:\n")
print("Engagement-focused ranking:")
print(posts.sort_values('score_engagement', ascending=False)[['post_id', 'score_engagement']].head(5))
print("\nQuality-focused ranking:")
print(posts.sort_values('score_quality', ascending=False)[['post_id', 'score_quality']].head(5))
print("\nViral-focused ranking:")
print(posts.sort_values('score_viral', ascending=False)[['post_id', 'score_viral']].head(5))

> **🧠 Meta Interview Signal**
>
> **Strong E5 Answer:** Understands that ML predicts probabilities, but the final ranking combines them with business weights. Mentions that changing weights doesn't require retraining.
>
> **E6 Answer Adds:** Discusses how to tune weights (grid search over A/B tests), how weights interact (increasing share weight may accidentally promote clickbait), and governance around weight changes.
>
> **Common Pitfall:** Thinking the model directly optimizes for "engagement." The model predicts individual actions; the value function defines what "engagement" means.

## 3. The Two Loops: Offline Training vs Online Serving

Every Meta ML system has two distinct operational modes:

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                            OFFLINE TRAINING LOOP                            │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│   ┌──────────┐    ┌──────────┐    ┌──────────┐    ┌──────────┐            │
│   │  Log     │───▶│  Label   │───▶│  Train   │───▶│  Eval    │            │
│   │  Data    │    │  Extract │    │  Model   │    │  Offline │            │
│   └──────────┘    └──────────┘    └──────────┘    └──────────┘            │
│                                                        │                   │
│   Latency: Hours to days                              │                   │
│   Scale: Billions of examples                          ▼                   │
│   Goal: Best possible model quality        ┌──────────────────┐           │
│                                            │  Model Registry  │           │
│                                            └──────────────────┘           │
└─────────────────────────────────────────────────────────────────────────────┘
                                                        │
                                                        │ Model Push
                                                        ▼
┌─────────────────────────────────────────────────────────────────────────────┐
│                            ONLINE SERVING LOOP                               │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│   ┌──────────┐    ┌──────────┐    ┌──────────┐    ┌──────────┐            │
│   │  User    │───▶│  Feature │───▶│  Model   │───▶│  Ranked  │            │
│   │  Request │    │  Fetch   │    │  Infer   │    │  Results │            │
│   └──────────┘    └──────────┘    └──────────┘    └──────────┘            │
│                                                                             │
│   Latency: <200ms total                                                    │
│   Scale: Millions of QPS                                                   │
│   Goal: Lowest latency with acceptable quality                             │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
```

### Key Differences

| Aspect | Offline Training | Online Serving |
|--------|------------------|----------------|
| **Latency tolerance** | Hours/days | Milliseconds |
| **Feature complexity** | Use all features | Only fast-to-compute features |
| **Data availability** | Full history | Real-time only |
| **Failure mode** | Delayed model update | User-visible degradation |
| **Scale metric** | Training throughput | QPS × latency |

### The Training-Serving Skew Problem

One of the most common production issues: **features behave differently in training vs serving**.

Examples:
- Feature computed from 7-day history in training, but only last hour available in serving
- Feature uses exact timestamp in training, but bucketed in serving for caching
- Feature joins user table in training, but user table has different freshness in serving

In [ ]:
# Demonstration: Training-Serving Skew

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

np.random.seed(42)
n_samples = 10000

# Training data: we have exact user_activity_count
user_activity_exact = np.random.poisson(50, n_samples)  # Exact count from logs
content_quality = np.random.normal(0, 1, n_samples)

# True click probability depends on both
logits = 0.02 * user_activity_exact + 0.5 * content_quality - 2
click_prob = 1 / (1 + np.exp(-logits))
clicks = np.random.binomial(1, click_prob)

# Train model with exact features
X_train = np.column_stack([user_activity_exact, content_quality])
model = LogisticRegression()
model.fit(X_train, clicks)

print(f"Training AUC (exact features): {roc_auc_score(clicks, model.predict_proba(X_train)[:, 1]):.4f}")

# Serving: we only have bucketed user_activity (simulating stale/approximate features)
# Bucket: 0-25, 25-50, 50-75, 75+
user_activity_bucketed = np.digitize(user_activity_exact, bins=[0, 25, 50, 75]) * 25

X_serve = np.column_stack([user_activity_bucketed, content_quality])
print(f"Serving AUC (bucketed features): {roc_auc_score(clicks, model.predict_proba(X_serve)[:, 1]):.4f}")
print(f"\n⚠️ AUC dropped due to feature skew!")

> **🧠 Meta Interview Signal**
>
> **Strong E5 Answer:** Can explain training-serving skew and give examples. Proposes solutions like feature logging at serving time, or using the same feature computation code.
>
> **E6 Answer Adds:** Discusses systematic approaches—feature stores with versioning, replay-based training, and monitoring for skew detection. Mentions specific failure cases they've debugged.
>
> **Common Pitfall:** Treating training and serving as the same environment. They're fundamentally different systems with different constraints.

## 4. Feature Freshness: A Meta Obsession

At Meta, **how fresh your features are** directly impacts model quality. The world changes fast:

- A post goes viral → engagement features spike
- User mood changes → preference signals shift  
- Breaking news happens → content relevance changes

```
Feature Freshness Spectrum
───────────────────────────────────────────────────────────────────────────────
                                                                              
 STATIC          DAILY           HOURLY          REAL-TIME                   
   │               │               │                │                        
   ▼               ▼               ▼                ▼                        
┌──────┐       ┌──────┐       ┌──────┐        ┌──────────┐                   
│User  │       │User  │       │Post  │        │Session   │                   
│demo- │       │long- │       │engage-│        │context   │                   
│graphics│      │term  │       │ment  │        │features  │                   
│      │       │prefs │       │stats │        │          │                   
└──────┘       └──────┘       └──────┘        └──────────┘                   
                                                                              
 Cheap to        Batch          Near-          Expensive                     
 compute         pipeline       real-time      to compute                    
                                streaming                                     
───────────────────────────────────────────────────────────────────────────────
```

In [ ]:
# Impact of feature freshness on prediction quality

def simulate_viral_post(hours_since_post, refresh_interval_hours):
    """
    Simulate a viral post and show how stale features hurt predictions.
    
    The post goes viral at hour 5, but if we only refresh features every N hours,
    our engagement prediction will be wrong.
    """
    # True engagement over time (post goes viral at hour 5)
    true_engagement = np.where(
        hours_since_post < 5,
        10 + hours_since_post * 2,  # Normal growth
        10 + 5 * 2 + (hours_since_post - 5) ** 2 * 10  # Viral explosion
    )
    
    # What our features show (updated every refresh_interval_hours)
    last_refresh = (hours_since_post // refresh_interval_hours) * refresh_interval_hours
    stale_engagement = np.where(
        last_refresh < 5,
        10 + last_refresh * 2,
        10 + 5 * 2 + (last_refresh - 5) ** 2 * 10
    )
    
    return true_engagement, stale_engagement

hours = np.arange(0, 24, 0.5)

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

for ax, refresh_interval in zip(axes, [1, 4, 12]):
    true_eng, stale_eng = simulate_viral_post(hours, refresh_interval)
    
    ax.plot(hours, true_eng, 'g-', label='True engagement', linewidth=2)
    ax.plot(hours, stale_eng, 'r--', label='Feature value (stale)', linewidth=2)
    ax.fill_between(hours, true_eng, stale_eng, alpha=0.3, color='red')
    ax.axvline(x=5, color='orange', linestyle=':', label='Goes viral')
    ax.set_xlabel('Hours since post')
    ax.set_ylabel('Engagement')
    ax.set_title(f'Refresh every {refresh_interval}h\n(Red area = prediction error)')
    ax.legend(loc='upper left', fontsize=8)

plt.tight_layout()
plt.show()

## 5. The Feedback Loop: Your System Affects Its Own Training Data

This is **the most important concept** for Meta ML systems:

**Your model's predictions determine what users see → which determines what they interact with → which becomes your training data → which trains your next model.**

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                          THE FEEDBACK LOOP                                  │
└─────────────────────────────────────────────────────────────────────────────┘

         ┌──────────────────┐
         │   User Opens     │
         │      App         │
         └────────┬─────────┘
                  │
                  ▼
         ┌──────────────────┐
         │  Model Ranks     │◀──────────────────────────────────┐
         │    Content       │                                   │
         └────────┬─────────┘                                   │
                  │                                             │
                  ▼                                             │
         ┌──────────────────┐                                   │
         │   User Sees      │                                   │
         │  Ranked Content  │                                   │
         └────────┬─────────┘                                   │
                  │                                             │
                  ▼                                             │
         ┌──────────────────┐                                   │
         │   User Takes     │                                   │
         │    Actions       │                                   │
         │  (click, like,   │                                   │
         │   hide, etc.)    │                                   │
         └────────┬─────────┘                                   │
                  │                                             │
                  ▼                                             │
         ┌──────────────────┐         ┌─────────────────┐      │
         │   Actions        │────────▶│   Model         │──────┘
         │   Logged         │         │   Retrained     │
         └──────────────────┘         └─────────────────┘


   DANGER: Content never shown = never trained on = never shown
           (The rich get richer, the invisible stay invisible)
```

In [ ]:
# Demonstration: Feedback loop leading to popularity bias

def simulate_feedback_loop(n_items=20, n_rounds=10, exploration_rate=0.0):
    """
    Simulate how a ranking system creates its own training data.
    
    Items have true quality, but the model only learns from shown items.
    Without exploration, it reinforces initial beliefs.
    """
    np.random.seed(42)
    
    # True quality of each item (unknown to model)
    true_quality = np.random.uniform(0, 1, n_items)
    
    # Initial model scores (random guess)
    model_scores = np.random.uniform(0, 1, n_items)
    
    exposure_counts = np.zeros(n_items)
    positive_feedback = np.zeros(n_items)
    
    history = []
    
    for round_num in range(n_rounds):
        # Rank by model score (with some exploration)
        if np.random.random() < exploration_rate:
            # Explore: show random item
            shown_item = np.random.randint(n_items)
        else:
            # Exploit: show highest scored item
            shown_item = np.argmax(model_scores)
        
        # User engagement depends on true quality
        engagement = np.random.random() < true_quality[shown_item]
        
        # Update model based on feedback
        exposure_counts[shown_item] += 1
        if engagement:
            positive_feedback[shown_item] += 1
        
        # Update model score (simple: fraction of positive feedback)
        for i in range(n_items):
            if exposure_counts[i] > 0:
                model_scores[i] = positive_feedback[i] / exposure_counts[i]
            # Items with no exposure keep their initial (random) score!
        
        history.append({
            'round': round_num,
            'shown_item': shown_item,
            'engagement': engagement,
            'correlation': np.corrcoef(true_quality, model_scores)[0, 1]
        })
    
    return history, exposure_counts, true_quality, model_scores

# Run simulation without exploration
history_no_explore, exposure_no_explore, true_q, model_s = simulate_feedback_loop(exploration_rate=0.0)

# Run simulation with exploration
history_explore, exposure_explore, _, _ = simulate_feedback_loop(exploration_rate=0.3)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Exposure distribution
axes[0].bar(range(20), exposure_no_explore, alpha=0.5, label='No exploration')
axes[0].bar(range(20), exposure_explore, alpha=0.5, label='30% exploration')
axes[0].set_xlabel('Item ID')
axes[0].set_ylabel('Times shown')
axes[0].set_title('Exposure Distribution After 10 Rounds')
axes[0].legend()

# Model-quality correlation over time
axes[1].plot([h['round'] for h in history_no_explore], 
             [h['correlation'] for h in history_no_explore], 
             label='No exploration', marker='o')
axes[1].plot([h['round'] for h in history_explore], 
             [h['correlation'] for h in history_explore], 
             label='30% exploration', marker='s')
axes[1].set_xlabel('Round')
axes[1].set_ylabel('Correlation (model score vs true quality)')
axes[1].set_title('Model Quality Over Time')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"Without exploration: {int(sum(exposure_no_explore == 0))} items NEVER shown")
print(f"With 30% exploration: {int(sum(exposure_explore == 0))} items NEVER shown")

> **🧠 Meta Interview Signal**
>
> **Strong E5 Answer:** Explains the feedback loop and why it leads to popularity bias. Proposes exploration (random traffic, epsilon-greedy) as a solution.
>
> **E6 Answer Adds:** Discusses exploration-exploitation tradeoff quantitatively, mentions specific techniques (Thompson sampling, contextual bandits), and connects to business impact (exploration has cost—lower short-term engagement).
>
> **Common Pitfall:** Ignoring the feedback loop entirely. Treating ML as a static prediction problem rather than a dynamic system.

## 6. Meta's ML System Principles: Summary

| Principle | What It Means | Why It Matters |
|-----------|--------------|----------------|
| **Everything is ranking** | Even classifiers produce scores for ranking | Unified architecture across products |
| **Multi-stage funnel** | Candidate gen → ranking → re-ranking | Handles scale (billions of items) |
| **Probability → value function** | Models predict P(action), weights define value | Separates ML from product decisions |
| **Training ≠ serving** | Different constraints, different optimizations | Prevents production surprises |
| **Feature freshness matters** | Stale features = wrong predictions | Real-time systems need real-time data |
| **Feedback loops exist** | Model predictions affect training data | Requires exploration, monitoring |
| **Offline ≠ online metrics** | AUC doesn't guarantee CTR improvement | Must validate with A/B tests |

---

## 🎯 Interview Drills

### Drill 1: "Explain the Meta ML architecture in 2 minutes"

**Model answer:**
> "At Meta, every ML system—Feed, Ads, Reels—follows a multi-stage funnel. First, candidate generation uses cheap retrieval to narrow billions of items to thousands. Then, a lightweight first-pass ranker reduces to hundreds. The main ranker—a deep neural network—precisely scores these with hundreds of features. Finally, a re-ranker applies business logic like diversity. Each stage trades off recall vs. precision and cost vs. quality. The models predict probabilities of user actions—click, like, share, hide—and a value function combines these into a final score. The weights in this function are product decisions, not learned. Key challenges are training-serving skew, feature freshness, and feedback loops where our predictions affect our future training data."

---

### Drill 2: "What tradeoff would you call out first?"

**Strong answers:**
- "Latency vs. model complexity. Heavier models are more accurate but slower. At Meta scale, 50ms extra means billions of lost impressions."
- "Feature freshness vs. compute cost. Real-time features are expensive but capture current state. We need to choose which features are worth the cost."
- "Exploration vs. exploitation. Showing suboptimal content hurts short-term engagement but is necessary to learn and avoid filter bubbles."

---

### Drill 3: "What would you change if latency doubles?"

**Strong answers:**
- "Reduce candidate set size—tighter filters in candidate generation"
- "Simplify ranking model—fewer layers, fewer features"
- "More aggressive caching of user/item embeddings"
- "Move computation offline—precompute more, personalize less at serving time"
- "Shard by geography—colocate users and content"

---

### Drill 4: "What breaks when traffic increases 10x?"

**Strong answers:**
- "Feature store becomes bottleneck—cache hit rates drop"
- "Model serving infrastructure—need more replicas, but cold start latency hurts"
- "Training pipeline—more logs to process, may fall behind"
- "Feature freshness degrades—streaming pipeline can't keep up"
- "A/B test velocity slows—need more traffic per experiment for significance"

---

## Next Steps

This notebook covered the **mental model**. The remaining notebooks dive deep:

1. **Problem Framing** → How to convert vague goals into ML objectives
2. **Data & Labels** → Where labels come from, biases in implicit feedback
3. **Feature Engineering** → What features matter, how to think about freshness
4. **Modeling & Tradeoffs** → Model architectures, multi-task learning
5. **Evaluation** → Offline vs online metrics, when they disagree
6. **Serving & Experimentation** → A/B testing, shadow deployment
7. **Monitoring & Feedback Loops** → Detecting drift, avoiding bias amplification